In [ ]:
from pathlib import Path
import pickle
from itertools import product
from smount_predictors.src.SeamountHelp import PipelinePredictor
from smount_predictors import SeamountHelp
import numpy as np
import pandas as pd
import simplekml
import xarray as xr


In [1]:
longitude_pairs = []
for lon in range(-180, 160, 20):
    longitude_pairs.append((lon, lon + 20))

latitude_pairs = []
for lat in range(-60, 70, 20):
    latitude_pairs.append((lat, lat + 20))


latlons = list(product(longitude_pairs, latitude_pairs))


[((-180, -160), (-60, -40)), ((-180, -160), (-40, -20)), ((-180, -160), (-20, 0)), ((-180, -160), (0, 20)), ((-180, -160), (20, 40)), ((-180, -160), (40, 60)), ((-180, -160), (60, 80)), ((-160, -140), (-60, -40)), ((-160, -140), (-40, -20)), ((-160, -140), (-20, 0)), ((-160, -140), (0, 20)), ((-160, -140), (20, 40)), ((-160, -140), (40, 60)), ((-160, -140), (60, 80)), ((-140, -120), (-60, -40)), ((-140, -120), (-40, -20)), ((-140, -120), (-20, 0)), ((-140, -120), (0, 20)), ((-140, -120), (20, 40)), ((-140, -120), (40, 60)), ((-140, -120), (60, 80)), ((-120, -100), (-60, -40)), ((-120, -100), (-40, -20)), ((-120, -100), (-20, 0)), ((-120, -100), (0, 20)), ((-120, -100), (20, 40)), ((-120, -100), (40, 60)), ((-120, -100), (60, 80)), ((-100, -80), (-60, -40)), ((-100, -80), (-40, -20)), ((-100, -80), (-20, 0)), ((-100, -80), (0, 20)), ((-100, -80), (20, 40)), ((-100, -80), (40, 60)), ((-100, -80), (60, 80)), ((-80, -60), (-60, -40)), ((-80, -60), (-40, -20)), ((-80, -60), (-20, 0)), ((-80

In [ ]:
model = pickle.load(open('out/cluster_tuned_model.pkl', 'rb'))
data_p = Path('data/swot_filtered.grd')

In [ ]:
redictions = pd.DataFrame(columns=['lon', 'lat', 'z', 'class', 'cluster'])
for zone in latlons:
    lon = zone[0]
    lat = zone[1]
    data = SeamountHelp.readAndFilterGRD(data_p, lon, lat).to_dataframe().reset_index()
    zone_pred = model.predict(data[['lon', 'lat', 'z']])
    predictions = pd.concat([predictions, zone_pred], axis=0)
global_predictions = xr.Dataset.from_dataframe(predictions.set_index(['lon', 'lat']))
global_predictions.to_netcdf('out/global_predictions.nc')

In [ ]:
mounts = predictions.groupby('cluster').mean().reset_index()
kml = simplekml.Kml()
for i, row in mounts.iterrows():
    kml.newpoint(name=f'{row.cluster}', coords=[(row.lon, row.lat, row.z)])
kml.save('out/global_mounts.kml')

In [ ]:
! open out/global_mounts.kml